In [1]:
import pandas as pd
import datasets
from datasets import Dataset
import numpy as np

In [2]:
df = pd.read_csv('train-balanced-sarcasm.csv')

In [3]:
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [4]:
# 90,000 datapoints for training and validation
# 10,000 datapoints for testing

# Second attempt: 50,000
df_bert = df[['label', 'comment']]
df_bert_subset = df_bert.sample(50000)
df_bert_subset['comment'] = df_bert_subset.comment.astype('str')
print(df_bert_subset.dtypes, "\n")
print("Subset: ", len(df_bert_subset[df_bert_subset['label'] == 1]), " ", len(df_bert_subset[df_bert_subset['label'] == 0]))

train, validate, test = np.split(df_bert_subset.sample(frac=1, random_state=42), 
                                 [int(.6*len(df_bert_subset)), int(.8*len(df_bert_subset))])
train = train.reset_index(drop = True)
validate = validate.reset_index(drop = True)
test = validate.reset_index(drop = True)
print('Train: ', len(train), ". Validate: ", len(validate), ". Test: ", len(test))
print('Train: ', len(train[train['label'] == 1]), " ", len(train[train['label'] == 0]), end = ". ")
print('Validate: ', len(validate[validate['label'] == 1]), " ", len(validate[validate['label'] == 0]), end = '. ')
print('Test: ', len(test[test['label'] == 1]), " ", len(test[test['label'] == 0]))

label       int64
comment    object
dtype: object 

Subset:  25373   24627
Train:  30000 . Validate:  10000 . Test:  10000
Train:  15183   14817. Validate:  5116   4884. Test:  5116   4884


In [5]:
train_dataset = Dataset.from_pandas(train)
validate_dataset = Dataset.from_pandas(validate)
test_dataset = Dataset.from_pandas(test)
dataset_dict = datasets.DatasetDict({"train" : train_dataset, "validate" : validate_dataset, "test" : test_dataset})

In [6]:
dataset_dict.save_to_disk('arrow-sarcasm-small-subset')

Saving the dataset (0/1 shards):   0%|          | 0/30000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]